In [39]:
import pandas as pd
import os
import requests
import py7zr
import xml.etree.ElementTree as ET
import pandas as pd

def process_xml(xml_file):
    label = xml_file[:xml_file.find('.')]
    tree = ET.parse('xml/' + xml_file)  # Replace with your actual file path
    root = tree.getroot()
    
    # Extract relevant data
    data = []
    for row in root.findall('row'):
        post_id = int(row.get('PostId'))
        creation_date = row.get('CreationDate')
        text = row.get('Text')
        data.append((post_id, creation_date, text))
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['PostId', 'CreationDate', 'Text'])
    
    # Convert CreationDate to datetime
    df['CreationDate'] = pd.to_datetime(df['CreationDate'])
    
    # Keep only the earliest entry per PostId
    df = df.sort_values(by=['PostId', 'CreationDate']).drop_duplicates(subset=['PostId'], keep='first')
    df['Label']=label
    df['Year'] = df.CreationDate.dt.year
    df['length'] = [len(str(text).strip()) for text in df.Text]
    
    df = df[(df.length>20) & (df.length<1000)]
    
    for year in [2018,2019,2020,2021,2022,2023]:
        if len(df[df.Year==year])>20:
            continue
        else:
            return None
    df = df[(df.Year>=2018) & (df.Year<=2023)].reset_index(drop=True)
    df = df[['Text','CreationDate','Year','Label']]
    return df
    
def extract_post(output_7z):
    # Define the URL and filenames
    url = "https://archive.org/download/stackexchange/" + output_7z
    original_xml = "PostHistory.xml"
    renamed_xml = output_7z[:output_7z.find('.stackexchange.com.7z')] + '.xml'
    
    # Download the .7z file
    print("Downloading the file " + output_7z)
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(output_7z, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print("Download complete.")
    else:
        print("Failed to download the file.")
        exit(1)
    
    # Extract the specific file
    print(f"Extracting {original_xml}...")
    with py7zr.SevenZipFile(output_7z, mode="r") as archive:
        all_files = archive.getnames()
        if original_xml in all_files:
            archive.extract(targets=[original_xml], path=".")
            print("Extraction complete.")
        else:
            print(f"{original_xml} not found in the archive.")
            exit(1)
    
    # Rename the extracted file
    if os.path.exists(original_xml):
        print("Renaming to " + renamed_xml)
        os.rename(original_xml, renamed_xml)
        print(f"Renamed {original_xml} to {renamed_xml}")
    
    # Optionally, remove the .7z file to save space
    os.remove(output_7z)
    print("Cleanup complete.")


In [17]:
#taken from https://archive.org/download/stackexchange 2025-02-05
df = pd.read_csv('tmp/stackexchange_dir.tsv', sep = '\t')
df_clean = []
for i,row  in df.iterrows():
    cur_name = row['Name']
    cur_size = row['Size']
    if '.stackexchange.com.7z' in cur_name and not('.meta.' in cur_name):
        cur_name = cur_name.strip()
        cur_name = cur_name[:cur_name.find('.7z')+3]
        if 'M' in cur_size:
            cur_size = cur_size[:-1]
            if float(cur_size)<50:
                df_clean.append([cur_name, cur_size])
df_clean = pd.DataFrame(df_clean, columns = ['name','size'])
df_clean.head()

,name,size
0,3dprinting.stackexchange.com.7z,18.9
1,ai.stackexchange.com.7z,35.3
2,anime.stackexchange.com.7z,36.5
3,astronomy.stackexchange.com.7z,49.0
4,avp.stackexchange.com.7z,20.9


In [19]:
for name in df_clean.name:
    print('*'*20)
    extract_post(name)

********************
Download complete.
Extracting PostHistory.xml...
Extraction complete.
Renaming to 3dprinting.xml
Renamed PostHistory.xml to 3dprinting.xml
Cleanup complete.
********************
Download complete.
Extracting PostHistory.xml...
Extraction complete.
Renaming to ai.xml
Renamed PostHistory.xml to ai.xml
Cleanup complete.
********************
Download complete.
Extracting PostHistory.xml...
Extraction complete.
Renaming to anime.xml
Renamed PostHistory.xml to anime.xml
Cleanup complete.
********************
Download complete.
Extracting PostHistory.xml...
Extraction complete.
Renaming to astronomy.xml
Renamed PostHistory.xml to astronomy.xml
Cleanup complete.
********************
Download complete.
Extracting PostHistory.xml...
Extraction complete.
Renaming to avp.xml
Renamed PostHistory.xml to avp.xml
Cleanup complete.
********************
Download complete.
Extracting PostHistory.xml...
Extraction complete.
Renaming to beer.xml
Renamed PostHistory.xml to beer.xml
Clea

In [47]:
xml_folder = "xml"  # Specify your folder path
files = [f for f in os.listdir(xml_folder) if os.path.isfile(os.path.join(xml_folder, f))]

df_final = []
for file in files:
    print(file)
    df_temp = process_xml(file)
    df_final.append(df_temp)

df_final = pd.concat(df_final).reset_index(drop = True)
df_final.to_csv('raw_stackexchange.csv', index = False)

3dprinting.xml
ai.xml
anime.xml
astronomy.xml
avp.xml
beer.xml
bioacoustics.xml
bioinformatics.xml
boardgames.xml
bricks.xml
buddhism.xml
cardano.xml
chess.xml
chinese.xml
civicrm.xml
coffee.xml
cogsci.xml
computergraphics.xml
conlang.xml
craftcms.xml
crafts.xml
cseducators.xml
cstheory.xml
devops.xml
drones.xml
earthscience.xml
ebooks.xml
economics.xml
elementaryos.xml
engineering.xml
eosio.xml
esperanto.xml
expatriates.xml
expressionengine.xml
fitness.xml
freelancing.xml
french.xml


C:\Users\jairo\AppData\Local\Temp\ipykernel_16064\628546003.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CreationDate'] = pd.to_datetime(df['CreationDate'])


gardening.xml
genealogy.xml
ham.xml
hardwarerecs.xml
health.xml
homebrew.xml
hsm.xml
interpersonal.xml
iot.xml
iota.xml
italian.xml
joomla.xml
korean.xml
langdev.xml
languagelearning.xml
latin.xml
lifehacks.xml
linguistics.xml
literature.xml
martialarts.xml
materials.xml
matheducators.xml
moderators.xml
monero.xml
musicfans.xml
mythology.xml
networkengineering.xml
opendata.xml
opensource.xml
or.xml
outdoors.xml
parenting.xml
patents.xml
pets.xml
pm.xml
poker.xml
portuguese.xml
proofassistants.xml
quantumcomputing.xml
retrocomputing.xml
reverseengineering.xml
russian.xml
scicomp.xml
sitecore.xml
solana.xml
sound.xml
spanish.xml
sports.xml
sqa.xml
stellar.xml
substrate.xml
sustainability.xml
tezos.xml
tor.xml
tridion.xml
ukrainian.xml
vegetarianism.xml
vi.xml
windowsphone.xml
woodworking.xml


In [51]:
len(df_final.Label.unique())

84

In [53]:
df_final.shape

(499359, 4)

In [57]:
df_final.groupby('Label').count()

,Text,CreationDate,Year
Label,,,
3dprinting,8257,8257,8257
ai,13690,13690,13690
anime,8257,8257,8257
astronomy,13089,13089,13089
avp,6535,6535,6535
...,...,...,...
tridion,2654,2654,2654
ukrainian,2544,2544,2544
vegetarianism,772,772,772


In [65]:
df_final.groupby(['Year']).count()

,Text,CreationDate,Label
Year,,,
2018,117132,117132,117132
2019,104621,104621,104621
2020,95920,95920,95920
2021,75136,75136,75136
2022,56804,56804,56804
2023,49746,49746,49746


In [61]:
df_final[df_final.Label=='ai'].groupby('Year').count()

,Text,CreationDate,Label
Year,,,
2018,1777,1777,1777
2019,2824,2824,2824
2020,3290,3290,3290
2021,2223,2223,2223
2022,1714,1714,1714
2023,1862,1862,1862
